In [1]:
from utils.predict import predict_multiple
from wandb import wandb
from keras.models import load_model

In [2]:
# Load model and weights from wandb
loaded_model = wandb.restore('model-best.h5', run_path="dat550/deepfake-efficientnet/runs/29425ckm")

model = load_model(loaded_model.name)

2022-05-14 12:33:59.227112: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
result = predict_multiple("./sample/test", model,frame_rate=1, plot=False)

100%|██████████| 10/10 [01:30<00:00,  9.04s/it]


In [4]:
result

,video_name,prediction,predicted_class,real_class
0,acifjvzvpm.mp4,0.430508,REAL,FAKE
1,acxnxvbsxk.mp4,0.619161,FAKE,FAKE
2,aagfhgtpmv.mp4,0.607483,FAKE,FAKE
3,abqwwspghj.mp4,0.835518,FAKE,FAKE
4,aczrgyricp.mp4,0.729519,FAKE,FAKE
5,acxwigylke.mp4,0.794970,FAKE,FAKE
6,abarnvbtwb.mp4,0.480931,REAL,REAL
7,acqfdwsrhi.mp4,0.023669,REAL,FAKE
